Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

1-hidden layer neural network with rectified linear units (nn.relu()) and 1024 hidden nodes. 

In [47]:
num_nodes= 1024
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_nodes]))
  biases_1 = tf.Variable(tf.zeros([num_nodes]))
  weights_2 = tf.Variable(
    tf.truncated_normal([num_nodes, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  relu_layer=tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)
  logits = tf.matmul(relu_layer, weights_2) + biases_2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels) + 0.001*tf.nn.l2_loss(weights_1) + 0.001*tf.nn.l2_loss(weights_2))

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
   tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1), weights_2) + biases_2)
  test_prediction =  tf.nn.softmax(
   tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1), weights_2) + biases_2)

In [48]:
num_steps = 3001

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 711.121216
Minibatch accuracy: 4.7%
Validation accuracy: 21.6%
Minibatch loss at step 500: 196.524872
Minibatch accuracy: 82.0%
Validation accuracy: 77.8%
Minibatch loss at step 1000: 115.746788
Minibatch accuracy: 80.5%
Validation accuracy: 81.2%
Minibatch loss at step 1500: 68.657089
Minibatch accuracy: 88.3%
Validation accuracy: 83.0%
Minibatch loss at step 2000: 41.163589
Minibatch accuracy: 93.0%
Validation accuracy: 85.0%
Minibatch loss at step 2500: 25.082373
Minibatch accuracy: 91.4%
Validation accuracy: 85.6%
Minibatch loss at step 3000: 15.466790
Minibatch accuracy: 86.7%
Validation accuracy: 86.3%
Test accuracy: 93.2%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [16]:
num_steps = 3001
batch_size = 128
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    #offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    offset = 0
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 322.365173
Minibatch accuracy: 11.7%
Validation accuracy: 25.8%
Minibatch loss at step 500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 64.0%
Minibatch loss at step 1000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 64.0%
Minibatch loss at step 1500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 64.0%
Minibatch loss at step 2000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 64.0%
Minibatch loss at step 2500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 64.0%
Minibatch loss at step 3000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 64.0%
Test accuracy: 70.7%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [25]:
num_nodes= 1024
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_nodes]))
  biases_1 = tf.Variable(tf.zeros([num_nodes]))
  weights_2 = tf.Variable(
    tf.truncated_normal([num_nodes, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  relu_layer=tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)
  dropouts = tf.nn.dropout(relu_layer, 0.5)
  logits = tf.matmul(dropouts, weights_2) + biases_2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
   tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1), weights_2) + biases_2)
  test_prediction =  tf.nn.softmax(
   tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1), weights_2) + biases_2)

In [26]:
num_steps = 3001
batch_size = 128
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    #offset = 0
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 435.166077
Minibatch accuracy: 17.2%
Validation accuracy: 28.8%
Minibatch loss at step 500: 41.423897
Minibatch accuracy: 75.0%
Validation accuracy: 78.9%
Minibatch loss at step 1000: 28.283506
Minibatch accuracy: 69.5%
Validation accuracy: 80.4%
Minibatch loss at step 1500: 16.334055
Minibatch accuracy: 81.2%
Validation accuracy: 79.9%
Minibatch loss at step 2000: 5.932854
Minibatch accuracy: 73.4%
Validation accuracy: 79.4%
Minibatch loss at step 2500: 4.461965
Minibatch accuracy: 70.3%
Validation accuracy: 79.8%
Minibatch loss at step 3000: 2.724249
Minibatch accuracy: 71.9%
Validation accuracy: 80.3%
Test accuracy: 87.7%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [28]:
################## build graph ####################
batch_size = 128
layer1_num_nodes = 1024
layer2_num_nodes = 1024
layer3_num_nodes = 1024
layer4_num_nodes = 128

graph = tf.Graph()
with graph.as_default():    
    ### data
    tf_train_datset = tf.placeholder(tf.float32, shape=(batch_size, image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_datset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    ### variables
    layer1_weights = tf.Variable(tf.truncated_normal([image_size * image_size, layer1_num_nodes], stddev=tf.sqrt(2.0/(image_size * image_size))))
    layer1_biases = tf.Variable(tf.zeros([layer1_num_nodes]))
    layer2_weights = tf.Variable(tf.truncated_normal([layer1_num_nodes, layer2_num_nodes], stddev=tf.sqrt(2.0/layer1_num_nodes)))
    layer2_biases = tf.Variable(tf.zeros([layer2_num_nodes]))
    layer3_weights = tf.Variable(tf.truncated_normal([layer2_num_nodes, layer3_num_nodes], stddev=tf.sqrt(2.0/layer2_num_nodes)))
    layer3_biases = tf.Variable(tf.zeros([layer3_num_nodes]))
    layer4_weights = tf.Variable(tf.truncated_normal([layer3_num_nodes, layer4_num_nodes], stddev=tf.sqrt(2.0/layer3_num_nodes)))
    layer4_biases = tf.Variable(tf.zeros([layer4_num_nodes]))
    output_weights = tf.Variable(tf.truncated_normal([layer4_num_nodes, num_labels], stddev=tf.sqrt(2.0/layer4_num_nodes)))
    output_biases = tf.Variable(tf.zeros([num_labels]))
    ### model forward propagation
    def model(data):
        layer1_output = tf.nn.relu(tf.matmul(data, layer1_weights) + layer1_biases)
        layer2_output = tf.nn.relu(tf.matmul(layer1_output, layer2_weights) + layer2_biases)
        layer3_output = tf.nn.relu(tf.matmul(layer2_output, layer3_weights) + layer3_biases)
        layer4_output = tf.nn.relu(tf.matmul(layer3_output, layer4_weights) + layer4_biases)    
        return tf.matmul(layer4_output, output_weights) + output_biases
    ### training computation
    logits = model(tf_train_datset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    ### optimizer
    learning_rate = 0.1
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
    ### predictions
    train_predictions = tf.nn.softmax(logits)
    valid_predictions = tf.nn.softmax(model(tf_valid_datset))
    test_predictions = tf.nn.softmax(model(tf_test_dataset))
    

In [29]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("initialized")
    for step in xrange(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_datset: batch_data, tf_train_labels: batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_predictions], feed_dict=feed_dict)
        if step % 500 == 0:
            print("Step : %d" % step)
            print("Loss : %f" % l)
            print("train accuracy : %f" % accuracy(predictions, batch_labels))
            print("valid accuracy : %f" % accuracy(valid_predictions.eval(), valid_labels))
    print("Test Accuracy : %f" % accuracy(test_predictions.eval(), test_labels))
        

initialized
Step : 0
Loss : 2.421656
train accuracy : 8.593750
valid accuracy : 25.220000
Step : 500
Loss : 0.358678
train accuracy : 87.500000
valid accuracy : 85.400000
Step : 1000
Loss : 0.481741
train accuracy : 85.937500
valid accuracy : 86.280000
Step : 1500
Loss : 0.262522
train accuracy : 92.187500
valid accuracy : 87.510000
Step : 2000
Loss : 0.242354
train accuracy : 93.750000
valid accuracy : 88.070000
Step : 2500
Loss : 0.312250
train accuracy : 90.625000
valid accuracy : 88.240000
Step : 3000
Loss : 0.344132
train accuracy : 86.718750
valid accuracy : 88.820000
Test Accuracy : 94.830000
